In [7]:
import os
from PIL import Image
import numpy as np
import json
import matplotlib.pyplot as plt
import glob
from tqdm import tqdm
from tifffile import TiffFile
import cv2
import tifffile

In [8]:
def showImagesHorizontally(list_of_files):
    plt.rcParams["figure.figsize"] = (18,10)
    fig = plt.figure()
    number_of_files = len(list_of_files)
    for i in range(number_of_files):
        a=fig.add_subplot(1,number_of_files,i+1)
        image = plt.imread(list_of_files[i])
        plt.imshow(image,cmap='Greys_r')
        plt.axis('off')
        
def showArraysHorizontally(list_of_arrs, names):
    plt.rcParams["figure.figsize"] = (18,10)
    fig = plt.figure()
    number_of_files = len(list_of_arrs)
    for i in range(number_of_files):
        a=fig.add_subplot(1,number_of_files,i+1)
        image = list_of_arrs[i]
        plt.title(names[i])
        plt.imshow(image,cmap='Greys_r')
        plt.axis('off')

In [9]:
root_path = '/datasets/EndoCV2022_ChallengeDataset/EAD2.0'
sub_dirs = os.listdir(root_path)

In [27]:
sub_dirs

['seq19_endocv22',
 'seq17_endocv22',
 'seq20_endocv22',
 'seq5_endocv22',
 'seq24_endocv22',
 'seq16_endocv22',
 'seq21_endocv22',
 'seq23_endocv22',
 'seq12_endocv22',
 'seq1_endocv22',
 'seq3_endocv22',
 'seq7_endocv22',
 'seq8_endocv22',
 'seq13_endocv22',
 'seq22_endocv22',
 'seq11_endocv22',
 'seq9_endocv22',
 'seq15_endocv22',
 'train.json',
 'seq10_endocv22',
 'seq18_endocv22',
 'val.json',
 'seq2_endocv22',
 'seq4_endocv22',
 'seq14_endocv22',
 'seq6_endocv22']

In [10]:
train_dirs = sub_dirs[:int(len(sub_dirs) * 0.8)]
val_dirs = sub_dirs[int(len(sub_dirs) * 0.8):]
len(train_dirs), len(val_dirs)

(20, 6)

In [22]:
# We need the following info for single instance in json: image_path, height, width, sem_seg_file_name (mask_path), add_subdir
save_dict = dict()
for sub_dir in tqdm(val_dirs):
    abs_path = os.path.join(root_path, sub_dir)
    images_dir = os.path.join(abs_path, 'images')
    mask_dir = os.path.join(abs_path, 'masks')
    images = os.listdir(images_dir)
    
    images = list(map(lambda x: os.path.join(images_dir, x), images))
    for image_path in images:
        record = {}
        image = cv2.imread(image_path)
        image_name = image_path.split('/')[-1].split('.jpg')[0]

        mask_path = image_path.replace('images', 'masks').replace('.jpg', '.tif')
        new_mask_path = mask_path.replace('.tif', '.png')
        mask = tifffile.imread(mask_path)
        , mask = cv2.threshold(mask, 127, 255, cv2.THRESH_BINARY)
        mask = (mask/255).astype(np.int32)

        # Convert to single mask with different labels for each class
        h, w = mask.shape[1], mask.shape[2]
        new_mask = np.zeros((h, w), dtype=np.int32)
        for idx in range(len(mask)):
            pixel_value = idx + 1
            new_mask = np.maximum(mask[idx] * pixel_value, new_mask)
            
        cv2.imwrite(new_mask_path, new_mask)
        record['file_name'] = image_path
        record['height'] = h
        record['width'] = w
        record['mask'] = new_mask_path
        save_dict[image_name] = record
    # Save into mask into .png format
    # showArraysHorizontally([mask[idx] for idx in range(5)], categorynames)

100%|██████████| 5/5 [00:15<00:00,  3.09s/it]


In [23]:
with open(os.path.join(root_path, 'val.json'), 'w') as f:
    json.dump(save_dict, f)